In [2]:
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

In [3]:
user_data = pd.read_csv('Data/user_data.csv', delimiter=';')
food_data = pd.read_csv('Data/food_data.csv', delimiter=';')

In [4]:
user_data_check = user_data.iloc[:, 20:]
count_non_zeros = user_data_check.astype(bool).sum(axis=1)

user_data_dup = pd.DataFrame()
columns_to_duplicate = user_data.columns[:20]

for i, row in user_data.iterrows():
    user_data_dup = user_data_dup.append([row[columns_to_duplicate]]*count_non_zeros[i])

user_data_dup = user_data_dup.reset_index(drop=True)

In [5]:
user_data_check

,643786,157963,638308,660128,638257,647311,638714,652332,652824,654460,...,680975,645863,715448,1697739,636682,653371,660820,634496,634476,649487
0,3,2,4,1,4,4,4,4,5,2,...,4,5,3,1,1,5,5,4,4,5
1,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
2,3,3,3,1,3,3,3,2,2,3,...,5,4,2,2,2,2,4,2,4,2
3,5,3,3,4,3,3,4,3,2,3,...,4,3,3,3,3,3,4,4,3,4
4,5,4,4,2,3,4,4,2,5,2,...,5,4,3,3,5,5,3,4,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,5,5,5,5,4,5,5,4,3,1,...,1,1,1,1,1,3,5,1,5,3
73,3,4,4,5,4,5,3,4,4,4,...,3,4,4,4,4,4,4,3,4,2
74,5,5,3,3,5,2,5,2,5,5,...,5,5,5,5,4,5,5,5,5,5
75,5,4,3,1,3,3,4,4,4,5,...,3,4,3,3,3,4,4,3,3,3


In [14]:
food_data_dup = pd.DataFrame()
rating = []

for i, row in user_data_check.iterrows():
    for j, item in enumerate(row):
        if item != 0:
            column_name = user_data_check.columns[j]
            rating.append(item)
            food_row = food_data[food_data['ID'] == int(column_name)]
            
            if not food_row.empty:
                food_data_dup = food_data_dup.append(food_row)

food_data_dup = food_data_dup.reset_index(drop=True)

In [15]:
food_data_dup

,ID,Name,Sweetness,Saltiness,Sourness,Bitterness,Savoriness,Fattiness,Spiciness,Calories,Fat,Saturated Fat,Carbohydrates,Net Carbohydrates,Sugar,Cholestrol,Sodium,Protein
0,643786,Fried Rice - Chinese comfort food,62.74,100.00,18.40,30.79,62.47,98.71,0.00000,422.10,18.88,5.87,46.90,43.58,2.20,144.54,1458.34,15.75
1,157963,Meatball Vegetable Soup,100.00,99.43,25.59,36.51,57.44,61.66,0.00000,552.13,17.68,6.37,76.31,70.99,31.84,80.96,850.41,22.41
2,638308,Chicken Satay,15.93,81.12,11.10,14.76,54.83,100.00,0.00000,833.70,72.88,30.22,19.50,14.80,7.41,122.50,1198.05,30.81
3,660128,Simple Rendang,10.46,100.00,13.41,16.59,70.23,56.21,20.25000,727.81,50.25,14.19,4.53,4.14,1.48,245.55,327.97,61.30
4,638257,Chicken Porridge,50.03,100.00,15.40,26.38,66.94,71.46,0.00000,393.05,6.53,1.58,53.46,51.34,7.27,57.00,826.49,27.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3434,653371,Nutella Stuffed French Toast,100.00,46.67,8.92,5.14,28.83,68.25,0.00000,377.81,20.32,15.04,40.27,37.84,35.55,178.09,131.14,8.83
3435,660820,Spaghetti With Bolognese Sauce,73.05,100.00,62.00,42.89,53.62,75.89,0.00000,346.41,27.17,11.04,7.64,5.97,3.74,76.09,749.20,2.12
3436,634496,BBQ Mac and Cheese,100.00,80.72,48.67,40.53,89.21,95.09,0.03675,812.67,25.59,13.88,109.20,104.26,17.21,70.09,1250.65,36.22
3437,634476,Bbq Chicken,21.30,100.00,13.34,16.50,69.85,55.59,0.00000,478.31,29.24,8.32,15.21,15.03,12.96,144.58,886.82,37.10


In [16]:
train_user_data = user_data_dup.drop(columns=['ID', 'Nama', 'Calories', 'Protein', 'Fat', 'Carbohydrates'])
train_food_data = food_data_dup.drop(columns=['ID', 'Name', 'Type', 'Link', 'Image_Type'])

In [17]:
train_user_data = pd.get_dummies(train_user_data, prefix=['Gender', 'Activity_Level'])

In [55]:
train_user_data

,Age,Height,Weight,Fruit,Healthy,Sweetness,Saltiness,Sourness,Bitterness,Savoriness,Fattiness,Spiciness,Gender_Female,Gender_Male,Active_Active,Active_Light activity,Active_Moderate active,Active_Not active,Active_Very active
0,25,165,44,2,2,1.716420,3.032389,1.057807,0.939549,2.007276,2.654416,0.011805,1,0,1,0,0,0,0
1,25,165,44,2,2,1.716420,3.032389,1.057807,0.939549,2.007276,2.654416,0.011805,1,0,1,0,0,0,0
2,25,165,44,2,2,1.716420,3.032389,1.057807,0.939549,2.007276,2.654416,0.011805,1,0,1,0,0,0,0
3,25,165,44,2,2,1.716420,3.032389,1.057807,0.939549,2.007276,2.654416,0.011805,1,0,1,0,0,0,0
4,25,165,44,2,2,1.716420,3.032389,1.057807,0.939549,2.007276,2.654416,0.011805,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3434,20,168,70,4,4,1.997396,3.682549,1.324011,1.223742,2.435298,3.151791,0.032304,0,1,1,0,0,0,0
3435,20,168,70,4,4,1.997396,3.682549,1.324011,1.223742,2.435298,3.151791,0.032304,0,1,1,0,0,0,0
3436,20,168,70,4,4,1.997396,3.682549,1.324011,1.223742,2.435298,3.151791,0.032304,0,1,1,0,0,0,0
3437,20,168,70,4,4,1.997396,3.682549,1.324011,1.223742,2.435298,3.151791,0.032304,0,1,1,0,0,0,0


In [18]:
train_food_data

,Sweetness,Saltiness,Sourness,Bitterness,Savoriness,Fattiness,Spiciness,Calories,Fat,Saturated Fat,Carbohydrates,Net Carbohydrates,Sugar,Cholestrol,Sodium,Protein
0,62.74,100.00,18.40,30.79,62.47,98.71,0.00000,422.10,18.88,5.87,46.90,43.58,2.20,144.54,1458.34,15.75
1,100.00,99.43,25.59,36.51,57.44,61.66,0.00000,552.13,17.68,6.37,76.31,70.99,31.84,80.96,850.41,22.41
2,15.93,81.12,11.10,14.76,54.83,100.00,0.00000,833.70,72.88,30.22,19.50,14.80,7.41,122.50,1198.05,30.81
3,10.46,100.00,13.41,16.59,70.23,56.21,20.25000,727.81,50.25,14.19,4.53,4.14,1.48,245.55,327.97,61.30
4,50.03,100.00,15.40,26.38,66.94,71.46,0.00000,393.05,6.53,1.58,53.46,51.34,7.27,57.00,826.49,27.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3434,100.00,46.67,8.92,5.14,28.83,68.25,0.00000,377.81,20.32,15.04,40.27,37.84,35.55,178.09,131.14,8.83
3435,73.05,100.00,62.00,42.89,53.62,75.89,0.00000,346.41,27.17,11.04,7.64,5.97,3.74,76.09,749.20,2.12
3436,100.00,80.72,48.67,40.53,89.21,95.09,0.03675,812.67,25.59,13.88,109.20,104.26,17.21,70.09,1250.65,36.22
3437,21.30,100.00,13.34,16.50,69.85,55.59,0.00000,478.31,29.24,8.32,15.21,15.03,12.96,144.58,886.82,37.10


In [19]:
# scale training data
item_train_unscaled = train_food_data
user_train_unscaled = train_user_data
rating = np.array(rating)
y_train_unscaled    = rating

scalerItem = StandardScaler()
scalerItem.fit(train_food_data)
item_train = scalerItem.transform(train_food_data)

scalerUser = StandardScaler()
scalerUser.fit(train_user_data)
user_train = scalerUser.transform(train_user_data)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(rating.reshape(-1, 1))
y_train = scalerTarget.transform(rating.reshape(-1, 1))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

True
True


In [20]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)

In [21]:


num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=num_outputs)
])

item_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=num_outputs)
])

input_user = tf.keras.layers.Input(shape=(19))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

input_item = tf.keras.layers.Input(shape=(16))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

output = tf.keras.layers.Dot(axes=1)([vu, vm])

model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 19)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 16)]         0           []                               
                                                                                                  
 sequential (Sequential)        (None, 32)           178592      ['input_1[0][0]']                
                                                                                                  
 sequential_1 (Sequential)      (None, 32)           47584       ['input_2[0][0]']                
                                                                                              

In [22]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt,
              loss=cost_fn)

In [23]:
model.fit([user_train, item_train], y_train, epochs=50)

Epoch 1/50
86/86 [==============================] - 5s 7ms/step - loss: 0.2666
Epoch 2/50
86/86 [==============================] - 0s 6ms/step - loss: 0.2352
Epoch 3/50
86/86 [==============================] - 0s 5ms/step - loss: 0.2250
Epoch 4/50
86/86 [==============================] - 0s 5ms/step - loss: 0.2167
Epoch 5/50
86/86 [==============================] - 0s 6ms/step - loss: 0.2094
Epoch 6/50
86/86 [==============================] - 1s 6ms/step - loss: 0.2034
Epoch 7/50
86/86 [==============================] - 0s 6ms/step - loss: 0.1963
Epoch 8/50
86/86 [==============================] - 0s 5ms/step - loss: 0.1909
Epoch 9/50
86/86 [==============================] - 0s 5ms/step - loss: 0.1863
Epoch 10/50
86/86 [==============================] - 1s 6ms/step - loss: 0.1821
Epoch 11/50
86/86 [==============================] - 0s 6ms/step - loss: 0.1783
Epoch 12/50
86/86 [==============================] - 0s 5ms/step - loss: 0.1749
Epoch 13/50
86/86 [==============================

In [94]:
#predict for index 1 test data
x = model.predict([user_test[34].reshape(1, 19), item_test[34].reshape(1, 16)])
#inverse transform the prediction
x = scalerTarget.inverse_transform(x)
x

1/1 [==============================] - 0s 28ms/step


array([[3.0516453]], dtype=float32)

In [79]:
x

array([[4.375146]], dtype=float32)

In [93]:
y = scalerTarget.inverse_transform(y_test[34].reshape(1, 1))
y

array([[1.]])

In [59]:
print(y_test[8])

[-1.]


In [110]:
model.save('recommendation_model.h5')

In [112]:
import pickle

with open('scalerUser.pkl', 'wb') as f:
    pickle.dump(scalerUser, f)

with open('scalerItem.pkl', 'wb') as f:
    pickle.dump(scalerItem, f)

In [102]:
scalerUser.inverse_transform(user_test)[1]

array([2.40000000e+01, 1.73000000e+02, 6.80000000e+01, 5.00000000e+00,
       4.00000000e+00, 1.68352000e+00, 3.10404444e+00, 1.14139778e+00,
       9.92208889e-01, 2.08114889e+00, 2.68727778e+00, 2.24645420e-02,
       1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [109]:
user_new = np.array([25,165,44,2,2,1.716420,3.032389,1.057807,0.939549,2.007276,2.654416,0.011805,1,0,1,0,0,0,0]).reshape(1, 19)
item_new = np.array([10.46,100,13.41,16.59,70.23,56.21,20.25,727.81,50.25,14.19,4.53,4.14,1.48,245.55,327.97,61.3]).reshape(1, 16)

user_new = scalerUser.transform(user_new)
item_new = scalerItem.transform(item_new)

rate = model.predict([user_new, item_new])

1/1 [==============================] - 0s 36ms/step


c:\Users\Bangkit\.conda\envs\tf210\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
c:\Users\Bangkit\.conda\envs\tf210\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


In [70]:
rate = scalerTarget.inverse_transform(rate)

In [71]:
rate

array([[3.4508297]], dtype=float32)